In [1]:
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

D:\Conda\envs\test_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running Model with RCIMP

In [9]:
f = FAIR(ch4_method="Thornhill2021")
f.define_time(1750, 2300, 1)  # start, end, step

"""
scenarios = [
    "high-extension",
    "high-overshoot",
    "medium-overshoot",
    "medium-extension",
    "low",
    "verylow",
    "verylow-overshoot",
]
"""

scenarios_Rcimp = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']
f.define_scenarios(scenarios_Rcimp)
fair_params_1_4_1_file = 'examples/data/calibrated_constrained_ensemble/calibrated_constrained_parameters_calibration1.4.1.csv'
df_configs = pd.read_csv(fair_params_1_4_1_file, index_col=0)
configs = df_configs.index  # this is used as a label for the "config" axis
f.define_configs(configs)
fair_species_configs_1_4_1_file = 'examples/data/calibrated_constrained_ensemble/species_configs_properties_calibration1.4.1.csv'
species_Rcimp, properties = read_properties(filename=fair_species_configs_1_4_1_file)
f.define_species(species_Rcimp, properties)
f.allocate()
"""
f.fill_from_csv(
    emissions_file='examples/data/calibrated_constrained_ensemble/extensions_1750-2500.csv',
    forcing_file='examples/data/calibrated_constrained_ensemble/volcanic_solar.csv',
)
"""
f.fill_from_rcmip()


f.forcing.sel(specie="Volcanic")
fill(
    f.forcing,
    f.forcing.sel(specie="Volcanic") * df_configs["forcing_scale[Volcanic]"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f.forcing,
    f.forcing.sel(specie="Solar") * df_configs["forcing_scale[Solar]"].values.squeeze(),
    specie="Solar",
)
f.fill_species_configs(fair_species_configs_1_4_1_file)
f.override_defaults(fair_params_1_4_1_file)
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)
initialise(f.ocean_heat_content_change, 0)
f.run()



Running 6728 projections in parallel: 100%|██████████| 550/550 [00:52<00:00, 10.45timesteps/s]


In [6]:
f2 = FAIR(ch4_method="Thornhill2021")
f2.define_time(1750, 2300, 1)  # start, end, step
scenarios_data = [
    "high-extension",
    "high-overshoot",
    "medium-overshoot",
    "medium-extension",
    "low",
    "verylow",
    "verylow-overshoot",
]
f2.define_scenarios(scenarios_data)
fair_params_1_4_1_file = 'examples/data/calibrated_constrained_ensemble/calibrated_constrained_parameters_calibration1.4.1.csv'
df_configs = pd.read_csv(fair_params_1_4_1_file, index_col=0)
configs = df_configs.index  # this is used as a label for the "config" axis
f2.define_configs(configs)
fair_species_configs_1_4_1_file = 'examples/data/calibrated_constrained_ensemble/species_configs_properties_calibration1.4.1.csv'
species_data, properties = read_properties(filename=fair_species_configs_1_4_1_file)
f2.define_species(species_data, properties)
f2.allocate()
f2.fill_from_csv(
    emissions_file='examples/data/calibrated_constrained_ensemble/extensions_1750-2500.csv',
    forcing_file='examples/data/calibrated_constrained_ensemble/volcanic_solar.csv',
)


f2.forcing.sel(specie="Volcanic")
fill(
    f2.forcing,
    f2.forcing.sel(specie="Volcanic") * df_configs["forcing_scale[Volcanic]"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f2.forcing,
    f2.forcing.sel(specie="Solar") * df_configs["forcing_scale[Solar]"].values.squeeze(),
    specie="Solar",
)
f2.fill_species_configs(fair_species_configs_1_4_1_file)
f2.override_defaults(fair_params_1_4_1_file)
initialise(f2.concentration, f2.species_configs["baseline_concentration"])
initialise(f2.forcing, 0)
initialise(f2.temperature, 0)
initialise(f2.cumulative_emissions, 0)
initialise(f2.airborne_emissions, 0)
initialise(f2.ocean_heat_content_change, 0)
f2.run()



Running 5887 projections in parallel: 100%|██████████| 550/550 [00:40<00:00, 13.50timesteps/s]


Exporting Maximum Forcing for each specie under each scenario

In [10]:
max_forcing_data_Rcimp = []
for scenario in scenarios_Rcimp:
    for specie in species_Rcimp:
        max_forcing = f.forcing.loc[dict(scenario=scenario, specie=specie)].max().item()
        max_forcing_data_Rcimp.append({
            'scenario': scenario,
            'specie': specie,
            'max_forcing': max_forcing
        })
max_forcing_df_Rcimp = pd.DataFrame(max_forcing_data_Rcimp)
max_forcing_df_Rcimp.to_csv('examples/data/calibrated_constrained_ensemble/max_forcing_Rcimp.csv')
max_forcing_df_Rcimp

,scenario,specie,max_forcing
0,ssp119,CO2 FFI,0.000000
1,ssp119,CO2 AFOLU,0.000000
2,ssp119,CO2,2.782390
3,ssp119,CH4,0.849355
4,ssp119,N2O,0.378282
...,...,...,...
483,ssp585,Ozone,1.176850
484,ssp585,Light absorbing particles on snow and ice,0.350233
485,ssp585,Stratospheric water vapour,0.269477
486,ssp585,Land use,0.012981


In [11]:
max_forcing_data = []
for scenario in scenarios_data:
    for specie in species_data:
        max_forcing = f2.forcing.loc[dict(scenario=scenario, specie=specie)].max().item()
        max_forcing_data.append({
            'scenario': scenario,
            'specie': specie,
            'max_forcing': max_forcing
        })
max_forcing_df = pd.DataFrame(max_forcing_data)
max_forcing_df.to_csv('examples/data/calibrated_constrained_ensemble/max_forcing_data.csv')
max_forcing_df

,scenario,specie,max_forcing
0,high-extension,CO2 FFI,0.000000
1,high-extension,CO2 AFOLU,0.000000
2,high-extension,CO2,13.129540
3,high-extension,CH4,1.394655
4,high-extension,N2O,0.804537
...,...,...,...
422,verylow-overshoot,Ozone,0.994862
423,verylow-overshoot,Light absorbing particles on snow and ice,0.284374
424,verylow-overshoot,Stratospheric water vapour,0.193880
425,verylow-overshoot,Land use,0.015461
